In [1]:
import pandas as pd
import json
from textblob import TextBlob
import operator
import re
import numpy as np
from sklearn import preprocessing

In [2]:
# import sys
# !{sys.executable} - m pip install textblob

In [3]:
df = pd.read_csv('../data/DomoArigatoData.csv') 

In [4]:
df

,Pathogenic?,Gene,history_class,full_history
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis..."
...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...


In [6]:
keylist = []
for index, row in pd.DataFrame(df['full_history']).itertuples():
    split_arr = row.split(" ; ")
    for keyval in split_arr:
        if keyval.count(':') == 1:
            if keyval.split(':')[0] not in keylist:
                keylist.append(keyval.split(':')[0])
        else:
            print(index, row)

In [7]:
keylist

['consent_approval',
 'ethnicity',
 'cancer_dx',
 'cancer_dx_type',
 'cancer_dx_age',
 'known_brca',
 'known_cancer',
 'rel_relation',
 'rel_cancer',
 'rel_age',
 '_method',
 'other_cancer',
 'relationships']

In [8]:
for key in keylist:
    df[key] = ""

In [9]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,known_cancer,rel_relation,rel_cancer,rel_age,_method,other_cancer,relationships
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,,,,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,,,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,,,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,,,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,


In [10]:
for index, value in pd.DataFrame(df['full_history']).itertuples():
    split_arr = value.split(" ; ")
    joe = [{x.split(":")[0]: x.split(":")[1]} for x in split_arr]
    for keyval in joe:
        col_name = list(keyval.keys())[0]
        val = list(keyval.values())[0]
        df.at[index, col_name] = val

In [11]:
rel_relation_list = []
for index, row in pd.DataFrame(df['rel_relation']).itertuples():
    try:
        for x in json.loads(row):
            rel_relation_list.append(x.lower())
    except:
        print(index, row)

419 
438 
439 
440 
1252 
1253 
1269 
1270 
1271 
1272 
1273 
1509 
1544 
2002 
2200 


In [12]:
rel_relation_list = list(set(list(filter(None, rel_relation_list))))

In [13]:
for i in range(len(rel_relation_list)):
    rel_relation_list[i]=TextBlob(rel_relation_list[i]).correct()

In [14]:
rel_relation_list = [str(x) for x in rel_relation_list]

In [15]:
for index, row in df.loc[df['relationships'] != ""].iterrows():
    parsed = row['relationships'].replace('=>', ':')
    box = json.loads(parsed)
    
    for i in range(len(box)):
        for item in box[i].items():
            if item[0]=='relationship_type':
                df.at[index, 'rel_relation'] = f'["{item[1]}"]'
            if item[0]=='cancer_dx_type':
                df.at[index, 'rel_cancer'] = f'["{item[1]}"]'
            if item[0]=='cancer_dx_age':
                df.at[index, 'rel_age'] = f'["{item[1]}"]'

In [16]:
for y in range(1, 4):
    df[f"cancer_dx_type_{str(y)}"] = ""
    df[f"cancer_dx_age_{str(y)}"] = ""
for i in range(1, 24):
    df[f"rel_{str(i)}_relationship"] = ""
    for t in range(1, 11):
        df[f"rel_{str(i)}_cancer_{str(t)}"] = ""
    for u in range(1, 4):
        df[f"rel_{str(i)}_age_{str(u)}"] = ""

In [17]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [18]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [19]:
rel_relation_search_list = ["mother","father","nephew","niece","uncle","aunt","brother","sister","grandmother","grandfather","great grandfather","great grandmother","cousin","dad","daughter","son","grand aunt","grand uncle","mom","great aunt","great uncle", "grandpa", "grandma"]
def find_probable_relation(test):
    test = test.lower()
    test = re.sub("([\(\[]).*?([\)\]])", "", test)
    if "side" in test:
        test = re.sub("(\w*'*\w)(\sside)", "", test)
    if "maternal" in test:
        test = test.replace("maternal", "")
    if "paternal" in test:
        test = test.replace("paternal", "")
    test = test.replace(" ", "")
    scores = {}
    for rel in rel_relation_search_list:
        scores[rel] = 1 - levenshteinDistance(test, rel)
    guess_relation = relation_dict[max(scores.items(), key=operator.itemgetter(1))[0]]
    df.at[index, f"rel_{i+1}_relationship"] = guess_relation

In [20]:
relation_dict = {
    'mother': 'mother',
    'mom': 'mother',
    'father': 'father',
    'dad': 'dad',
    'nephew': 'nephew',
    'niece': 'niece',
    'uncle': 'uncle',
    'aunt': 'aunt',
    'brother': 'brother',
    'sister': 'sister',
    'cousin': 'cousin',
    'grandmother': 'grandmother',
    'grandfather': 'grandfather',
    'great grandfather': 'great grandfather',
    'great grandmother': 'great grandmother',
    'daughter': 'daughter',
    'son': 'son',
    'grand aunt': 'great aunt',
    'grand uncle': 'great uncle',
    'great aunt': 'great aunt',
    'great uncle': 'great uncle',
    'grandpa': 'grandfather',
    'grandma': 'grandmother'
}

In [21]:
def find_probable_age(test, index, rel_index):
    test = re.findall("[0-9]+", test)
    for i in range(len(test)):
        df.at[index, f"rel_{rel_index+1}_age_{i+1}"] = test[i]

In [22]:
rel_cancer_search_list = ['vulvar','lymphoma','ovarian','brain','breast','prostate','blood','bone','abdominal','leukemia','lung','ampulla','skin','anal','colon','thyroid','appendix','lympnodes','basal','bile duct','bladder','intestines','liver','bowel','sinus','spinal','uterine','cervical','pancreas','cheek','ciliary','voice box','endometrial','rectum','tonsil','tongue','esophageal','thrombocytopenia','testicles','stomach','squamous','throat','retina','renal','myeloma', 'melanoma', 'oral','muscle','polyps','kidney','gall bladder','jaw','heart','glaucoma','glioblastoma','duodenum']

In [23]:
def find_probable_cancer(test, index, rel_index):
    test = test.lower()
    if test == "?" or test == "unknown":
        pass
    else:
        test = re.split(",|/|;", test)
        test = [re.sub("([\(\[]).*?([\)\]])", "", x) for x in test]
        for i in range(len(test)):
            scores = {}
            for rel in rel_cancer_search_list:
                scores[rel] = 1 - levenshteinDistance(test[i], rel)
            guess_cancer = max(scores.items(), key=operator.itemgetter(1))[0]
            df.at[index, f"rel_{rel_index+1}_cancer_{i+1}"] = guess_cancer

In [24]:
for index, row in df[['rel_relation', 'rel_cancer', 'rel_age']].iterrows():
    relations = []
    cancers = []
    ages = []
    if row['rel_relation'] != "":
        relations = json.loads(row['rel_relation'])
    else:
        pass
    if row['rel_cancer'] != "":
        cancers = json.loads(row['rel_cancer'])
    else:
        pass
    if row['rel_age'] != "":
        ages = json.loads(row['rel_age'])
    else:
        pass
    
    for i in range(len(relations)):
        if relations[i] == "":
            pass
        else:
            guess_relation = find_probable_relation(relations[i])
            guess_cancer = find_probable_cancer(cancers[i], index, i)
            guess_age = find_probable_age(ages[i], index, i)


In [25]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [26]:
for index, row in df.loc[df['cancer_dx_type'] != ""].iterrows():
    test = row['cancer_dx_type'].lower()
    test = test.replace("precancer", "")
    test = test.replace("pre-cancer", "")
    if test == "na" or test == "n/a":
        pass
    else:
        test = test.split(',')
        for i in range(len(test)):
            scores = {}
            for rel in rel_cancer_search_list:
                scores[rel] = 1 - levenshteinDistance(test[i], rel)
            guess_cancer = max(scores.items(), key=operator.itemgetter(1))[0]
            df.at[index, f"cancer_dx_type_{i+1}"] = guess_cancer

In [27]:
for index, row in df.loc[df['cancer_dx_age'] != ""].iterrows():
    test = re.findall("[0-9]+", row['cancer_dx_age'])
    for i in range(len(test)):
        df.at[index, f"cancer_dx_age_{i+1}"] = test[i]

In [28]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [63]:
# df2 = pd.read_csv('data/Ethnicity.csv')
# adjectives = [x.replace(u'\xa0', u'').lower() for x in list(df2['Adjective'])]
# race = [x.replace(u'\xa0', u'') for x in list(df2['Race'])]
# final = []
# final.extend([list(a) for a in zip(adjectives, race)])
# ethinicities = dict(final)
# data = [TextBlob(x.lower().strip().replace('','')).correct() for x in list(df['ethnicity'])]

for i in range(len(data)):
    scores = {}
    for rel in list(ethinicities.keys()):
            scores[rel] = 1 - levenshteinDistance(data[i], rel)
    ethnicity_guess = ethinicities[max(scores.items(), key=operator.itemgetter(1))[0]]
    df.at[i, 'ethnicity'] = ethnicity_guess

In [64]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,White,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,White,yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Native American,yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,no,...,,,,,,,,,,


In [65]:
df['Gene'] = df['Gene'].fillna('')
df['history_class'] = df['history_class'].replace('not_strong ', 'not_strong')
df = df.drop(columns=['consent_approval'])
df['cancer_dx']=df['cancer_dx'].replace('Yes','yes').replace('No', 'no').map(dict(yes=1, no=0))
df[df['cancer_dx'].isnull()]['cancer_dx'][399]=1
df['cancer_dx']=df['cancer_dx'].fillna(0)
df = df.drop(columns=['_method'])
df = df.drop(columns=['known_cancer'])
df['known_brca']=df['known_brca'].fillna('unknown')
df['known_brca']=df['known_brca'].replace('Yes','yes').replace('No', 'no').replace('U', 'unknown').map(dict(yes=1, no=0, unknown=-1))

In [66]:
df

,Pathogenic?,Gene,history_class,full_history,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,rel_relation,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,White,1.0,Pre-Cancer Vulvar Due to HPV,57,-1.0,"[""Mother"", ""Grandmother Mothers Side"", ""Grandf...",...,,,,,,,,,,
1,False,,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,White,0.0,Melanoma,32,-1.0,"["""", """", """"]",...,,,,,,,,,,
2,False,,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,Hispanic,1.0,Breast,37,-1.0,"[""Father"", ""Sister"", ""Aunt""]",...,,,,,,,,,,
3,False,,strong_personal,ethnicity:english extraction caucasion ; cance...,White,1.0,Prostate,55,-1.0,"[""father"", ""uncle"", """"]",...,,,,,,,,,,
4,False,,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",White,1.0,uterine,58,-1.0,"[""brother"", ""brother"", ""female cousin"", ""mater...",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,-1.0,"["""", """", """"]",...,,,,,,,,,,
2209,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,-1.0,"[""Grandfather"", """", """"]",...,,,,,,,,,,
2210,False,,none,_method:patch ; consent_approval:yes ; ethnici...,Native American,1.0,Cervical carcinoma in situ,22,-1.0,"["""", """", """"]",...,,,,,,,,,,
2211,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,0.0,"["""", """", """"]",...,,,,,,,,,,


In [100]:
df.to_csv('../data/cleaned.csv', index = False)

In [107]:
################################

In [101]:
labelencoder = preprocessing.LabelEncoder()

In [28]:
df = pd.read_csv('../data/cleaned.csv')

C:\Users\katsu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (16,18,22,23,24,25,26,27,28,29,30,37,38,50,51,52,53,54,64,65,78,90,91,92,104,105,106,118,119,120,132,133,134,146,147,160,161,174,175,188,189,202,203,216,217,230,231,244,245,258,259,272,273,286,287) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
# df = df.drop(columns = ['Gene', 'full_history', 'rel_relation', 'rel_cancer', 'rel_age', 'relationships', 'cancer_dx_type', 'cancer_dx_age', 'other_cancer'])
df = df.drop(columns = ['full_history', 'rel_relation', 'rel_cancer', 'rel_age', 'relationships', 'cancer_dx_type', 'cancer_dx_age', 'other_cancer'])

In [30]:
df

,Pathogenic?,Gene,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,White,1.0,-1.0,vulvar,57.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,NaN,strong_personal,White,0.0,-1.0,melanoma,32.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,NaN,strong_personal,Hispanic,1.0,-1.0,breast,37.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,NaN,strong_personal,White,1.0,-1.0,prostate,55.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,NaN,strong_personal,White,1.0,-1.0,uterine,58.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,White,0.0,-1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2209,False,NaN,none,White,0.0,-1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210,False,NaN,none,Native American,1.0,-1.0,cervical,22.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2211,False,NaN,none,White,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df.replace(np.nan, "?")

In [32]:
df

,Pathogenic?,Gene,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,?,strong_personal,White,1.0,-1,vulvar,57,?,?,...,?,?,?,?,?,?,?,?,?,?
1,False,?,strong_personal,White,0.0,-1,melanoma,32,?,?,...,?,?,?,?,?,?,?,?,?,?
2,False,?,strong_personal,Hispanic,1.0,-1,breast,37,?,?,...,?,?,?,?,?,?,?,?,?,?
3,False,?,strong_personal,White,1.0,-1,prostate,55,?,?,...,?,?,?,?,?,?,?,?,?,?
4,False,?,strong_personal,White,1.0,-1,uterine,58,?,?,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,?,none,White,0.0,-1,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2209,False,?,none,White,0.0,-1,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2210,False,?,none,Native American,1.0,-1,cervical,22,?,?,...,?,?,?,?,?,?,?,?,?,?
2211,False,?,none,White,0.0,0,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


In [33]:
df['known_brca'].replace(-1, "unknown", inplace = True)
df['known_brca'].replace(0, "no", inplace = True)
df['known_brca'].replace(1, "yes", inplace = True)
df['cancer_dx'].replace(0, "no", inplace = True)
df['cancer_dx'].replace(1, "yes", inplace = True)

In [34]:
df

,Pathogenic?,Gene,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,?,strong_personal,White,yes,unknown,vulvar,57,?,?,...,?,?,?,?,?,?,?,?,?,?
1,False,?,strong_personal,White,no,unknown,melanoma,32,?,?,...,?,?,?,?,?,?,?,?,?,?
2,False,?,strong_personal,Hispanic,yes,unknown,breast,37,?,?,...,?,?,?,?,?,?,?,?,?,?
3,False,?,strong_personal,White,yes,unknown,prostate,55,?,?,...,?,?,?,?,?,?,?,?,?,?
4,False,?,strong_personal,White,yes,unknown,uterine,58,?,?,...,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,?,none,White,no,unknown,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2209,False,?,none,White,no,unknown,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
2210,False,?,none,Native American,yes,unknown,cervical,22,?,?,...,?,?,?,?,?,?,?,?,?,?
2211,False,?,none,White,no,no,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


In [36]:
df.to_csv('../data/supertml_input.csv', index = False)

In [35]:
#############################

In [118]:
pattern = r'rel_[0-9]_cancer'
pattern2 = r'rel_[0-9][0-9]_cancer'
pattern3 = r'cancer_dx_type_'
lis = [i for i, x in enumerate(df.columns) if re.search(pattern, x)]
lis2 = [i for i, x in enumerate(df.columns) if re.search(pattern2, x)]
lis3 = [i for i, x in enumerate(df.columns) if re.search(pattern3, x)]
cancerallcolumns = []
cancerallcolumns.extend(lis)
cancerallcolumns.extend(lis2)
cancerallcolumns.extend(lis3)
pattern = r'rel_[0-9]_relationship'
pattern2 = r'rel_[0-9][0-9]_relationship'
lis = [i for i, x in enumerate(df.columns) if re.search(pattern, x)]
lis2 = [i for i, x in enumerate(df.columns) if re.search(pattern2, x)]
relationshipsallcolumns = []
relationshipsallcolumns.extend(lis)
relationshipsallcolumns.extend(lis2)
uniquecancer = list(pd.unique(df.iloc[:, cancerallcolumns].values.ravel('K')))
uniquerelationships = list(pd.unique(df.iloc[:, relationshipsallcolumns].values.ravel('K')))
uniquecancer.extend(' ')
uniquerelationships.extend(' ')
lecancer = preprocessing.LabelEncoder()
lerelationships = preprocessing.LabelEncoder()
lerelationships.fit(uniquerelationships)
lecancer.fit(uniquecancer)
df.iloc[:, cancerallcolumns] = df.iloc[:, cancerallcolumns].fillna(' ').apply(lecancer.transform)
df.iloc[:, relationshipsallcolumns] = df.iloc[:, relationshipsallcolumns].fillna(' ').apply(lerelationships.transform)
df['history_class']=df['history_class'].astype('category').cat.codes
df['Pathogenic?']=df['Pathogenic?'].astype('category').cat.codes
df['ethnicity']=df['ethnicity'].astype('category').cat.codes

for x in range(len(df.columns)):
    if 'age' in df.columns[x]:
        df[df.columns[x]].replace(np.nan, -1, inplace = True)
    elif 'known_brca' in df.columns[x]:
        df[df.columns[x]].replace(np.nan, -1, inplace = True)

In [119]:
df

,Pathogenic?,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,cancer_dx_type_3,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,0,3,6,1.0,-1.0,57,57.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
1,0,3,6,0.0,-1.0,32,32.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
2,0,3,2,1.0,-1.0,12,37.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
3,0,3,6,1.0,-1.0,40,55.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
4,0,3,6,1.0,-1.0,55,58.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,0,0,6,0.0,-1.0,0,-1.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
2209,0,0,6,0.0,-1.0,0,-1.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
2210,0,0,5,1.0,-1.0,13,22.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0
2211,0,0,6,0.0,0.0,0,-1.0,0,-1.0,0,...,0,0,0,0,0,0,0,-1.0,-1.0,-1.0


In [120]:
df.to_csv('normalized_one_hot.csv', index = False)

In [58]:
non_age = [df.columns[x] for x in range(len(df.columns)) if 'age' not in df.columns[x]]
to_be_removed = {'cancer_dx', 'known_brca'}
str_col = [e for e in non_age if e not in to_be_removed]
str_col

['Pathogenic?',
 'history_class',
 'ethnicity',
 'cancer_dx_type_1',
 'cancer_dx_type_2',
 'cancer_dx_type_3',
 'rel_1_relationship',
 'rel_1_cancer_1',
 'rel_1_cancer_2',
 'rel_1_cancer_3',
 'rel_1_cancer_4',
 'rel_1_cancer_5',
 'rel_1_cancer_6',
 'rel_1_cancer_7',
 'rel_1_cancer_8',
 'rel_1_cancer_9',
 'rel_1_cancer_10',
 'rel_2_relationship',
 'rel_2_cancer_1',
 'rel_2_cancer_2',
 'rel_2_cancer_3',
 'rel_2_cancer_4',
 'rel_2_cancer_5',
 'rel_2_cancer_6',
 'rel_2_cancer_7',
 'rel_2_cancer_8',
 'rel_2_cancer_9',
 'rel_2_cancer_10',
 'rel_3_relationship',
 'rel_3_cancer_1',
 'rel_3_cancer_2',
 'rel_3_cancer_3',
 'rel_3_cancer_4',
 'rel_3_cancer_5',
 'rel_3_cancer_6',
 'rel_3_cancer_7',
 'rel_3_cancer_8',
 'rel_3_cancer_9',
 'rel_3_cancer_10',
 'rel_4_relationship',
 'rel_4_cancer_1',
 'rel_4_cancer_2',
 'rel_4_cancer_3',
 'rel_4_cancer_4',
 'rel_4_cancer_5',
 'rel_4_cancer_6',
 'rel_4_cancer_7',
 'rel_4_cancer_8',
 'rel_4_cancer_9',
 'rel_4_cancer_10',
 'rel_5_relationship',
 'rel_5_ca

In [165]:
cancer_cols = [df.columns[x] for x in range(len(df.columns)) if 'cancer' in df.columns[x] and 'age' not in df.columns[x]]

In [166]:
cancer_cols.remove('cancer_dx')

In [167]:
cancer_cols

['cancer_dx_type_1',
 'cancer_dx_type_2',
 'cancer_dx_type_3',
 'rel_1_cancer_1',
 'rel_1_cancer_2',
 'rel_1_cancer_3',
 'rel_1_cancer_4',
 'rel_1_cancer_5',
 'rel_1_cancer_6',
 'rel_1_cancer_7',
 'rel_1_cancer_8',
 'rel_1_cancer_9',
 'rel_1_cancer_10',
 'rel_2_cancer_1',
 'rel_2_cancer_2',
 'rel_2_cancer_3',
 'rel_2_cancer_4',
 'rel_2_cancer_5',
 'rel_2_cancer_6',
 'rel_2_cancer_7',
 'rel_2_cancer_8',
 'rel_2_cancer_9',
 'rel_2_cancer_10',
 'rel_3_cancer_1',
 'rel_3_cancer_2',
 'rel_3_cancer_3',
 'rel_3_cancer_4',
 'rel_3_cancer_5',
 'rel_3_cancer_6',
 'rel_3_cancer_7',
 'rel_3_cancer_8',
 'rel_3_cancer_9',
 'rel_3_cancer_10',
 'rel_4_cancer_1',
 'rel_4_cancer_2',
 'rel_4_cancer_3',
 'rel_4_cancer_4',
 'rel_4_cancer_5',
 'rel_4_cancer_6',
 'rel_4_cancer_7',
 'rel_4_cancer_8',
 'rel_4_cancer_9',
 'rel_4_cancer_10',
 'rel_5_cancer_1',
 'rel_5_cancer_2',
 'rel_5_cancer_3',
 'rel_5_cancer_4',
 'rel_5_cancer_5',
 'rel_5_cancer_6',
 'rel_5_cancer_7',
 'rel_5_cancer_8',
 'rel_5_cancer_9',
 '

In [174]:
cancer_map = {cancer: i for i, cancer in enumerate(pd.unique(df[cancer_cols].values.ravel('K')))}

In [173]:
#will work with cleaned
pd.unique(df[cancer_cols].values.ravel('K'))

array(['vulvar', 'melanoma', 'breast', 'prostate', 'uterine',
       'intestines', 'kidney', 'colon', 'skin', nan, 'lymphoma',
       'thyroid', 'endometrial', 'basal', 'cervical', 'squamous',
       'bile duct', 'glioblastoma', 'testicles', 'rectum', 'glaucoma',
       'ovarian', 'lung', 'tonsil', 'pancreas', 'bone', 'stomach',
       'leukemia', 'bladder', 'ampulla', 'myeloma', 'throat',
       'gall bladder', 'liver', 'renal', 'brain', 'esophageal',
       'thrombocytopenia', 'jaw', 'anal', 'lympnodes', 'appendix', 'oral',
       'voice box', 'tongue', 'duodenum', 'polyps', 'cheek', 'abdominal',
       'sinus', 'spinal', 'retina', 'ciliary', 'blood', 'muscle', 'heart',
       'bowel'], dtype=object)

In [175]:
cancer_map

{'abdominal': 48,
 'ampulla': 29,
 'anal': 39,
 'appendix': 41,
 'basal': 13,
 'bile duct': 16,
 'bladder': 28,
 'blood': 53,
 'bone': 25,
 'bowel': 56,
 'brain': 35,
 'breast': 2,
 'cervical': 14,
 'cheek': 47,
 'ciliary': 52,
 'colon': 7,
 'duodenum': 45,
 'endometrial': 12,
 'esophageal': 36,
 'gall bladder': 32,
 'glaucoma': 20,
 'glioblastoma': 17,
 'heart': 55,
 'intestines': 5,
 'jaw': 38,
 'kidney': 6,
 'leukemia': 27,
 'liver': 33,
 'lung': 22,
 'lymphoma': 10,
 'lympnodes': 40,
 'melanoma': 1,
 'muscle': 54,
 'myeloma': 30,
 nan: 9,
 'oral': 42,
 'ovarian': 21,
 'pancreas': 24,
 'polyps': 46,
 'prostate': 3,
 'rectum': 19,
 'renal': 34,
 'retina': 51,
 'sinus': 49,
 'skin': 8,
 'spinal': 50,
 'squamous': 15,
 'stomach': 26,
 'testicles': 18,
 'throat': 31,
 'thrombocytopenia': 37,
 'thyroid': 11,
 'tongue': 44,
 'tonsil': 23,
 'uterine': 4,
 'voice box': 43,
 'vulvar': 0}